In [23]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
import sys,os,shutil
sys.path.append("..")
import argparse
import logging
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext

from snowflake import connector
from os import getenv
from collections import namedtuple

from datetime import datetime
from openpyxl import load_workbook
from collections import Counter

logging.basicConfig(level=logging.ERROR)

from dotenv import load_dotenv
load_dotenv()


# from common.util.database.snowflake import SnowflakeLoader
# from common.loader.excel_loader.util import source_to_parquet
# from common.constant.mode import ETLMode
# license_hp_service = 'C:/cloud_service'
# sys.path.append(license_hp_service)
# import hoangphuc__function


True

In [24]:
def download_from_sharepoint(account_sharepoint, pwsd_sharepoint, site_url, file_name, file_path, download_to):
    try:
        # credentials = ClientCredential(account_sharepoint, pwsd_sharepoint)
        # ctx = ClientContext(site_url).with_credentials(credentials)

        ctx = ClientContext(site_url).with_credentials(UserCredential(account_sharepoint, pwsd_sharepoint))
        _file = open(download_to, "wb")
        ctx.web.get_file_by_server_relative_path(file_path).download(
                _file
            ).execute_query()
        print(f"====Downloaded Source File for {file_name}====")
        _file.close()
    except Exception as e:
        print(e)
        input("Cannot download. Press any key to exit.")

def create_folder_if_not_exists(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
 
def del_file_in_folder(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

def ask__yes_no(mess):
    while True:
        answer = input('\n\t{} y/n: '.format(mess))
        if answer in ('y','n'):
            break
        else:
            print("\tInvalid input")
    return answer


In [25]:
report_folder__cdm = 'c:/cdm/'
create_folder_if_not_exists(report_folder__cdm)
del_file_in_folder(report_folder__cdm)
 
import_snowflake = 'C:/import_snowflake/'
create_folder_if_not_exists(import_snowflake)


In [26]:
raw_lists = [
"/sites/BAS/Shared Documents/4.CDM/data-requirements/cdm__quarterly_data.csv",
"/sites/BAS/Shared Documents/4.CDM/data-requirements/cdm__seller_code_mapping.csv",
"/sites/BAS/Shared Documents/4.CDM/data-requirements/cdm__weekly_data.csv",
"/sites/BAS/Shared Documents/4.CDM/data-requirements/certificate_target.csv",
"/sites/BAS/Shared Documents/4.CDM/data-requirements/point_target.csv"
]


In [27]:
site_url = "https://avenzacorp.sharepoint.com/sites/BAS/"
account_sharepoint = getenv('account_sharepoint')
pwsd_sharepoint = getenv('pwsd_sharepoint')

print(f"""
====your account = {account_sharepoint}
====your password = {pwsd_sharepoint}
""")
# account_sharepoint = input("Pls input your company account ex. phucnguyen@synagie.com: ")
# pwsd_sharepoint = input("Pls input your password: ")

download_folder = report_folder__cdm

answer = ask__yes_no("Do you want do re-download from sharepoint? ")
if answer == 'y':
    create_folder_if_not_exists(download_folder)    
    del_file_in_folder(download_folder)
 
    for file_path in raw_lists:
        file_name = file_path.split("/")[-1] 
        print(f'\n{file_name} is downloading...')

        download_to = os.path.join(download_folder, file_name)
        download_from_sharepoint(account_sharepoint, pwsd_sharepoint, site_url, file_name, file_path, download_to)



====your account = nhoang@synagie.com
====your password = Ibethere4u@188


cdm__quarterly_data.csv is downloading...
====Downloaded Source File for cdm__quarterly_data.csv====

cdm__seller_code_mapping.csv is downloading...
====Downloaded Source File for cdm__seller_code_mapping.csv====

cdm__weekly_data.csv is downloading...
====Downloaded Source File for cdm__weekly_data.csv====

certificate_target.csv is downloading...
====Downloaded Source File for certificate_target.csv====

point_target.csv is downloading...
====Downloaded Source File for point_target.csv====


In [13]:
class SnowflakeLoader:
   def __init__(self, schema, target_table):
       self.schema = schema.upper()
       self.cred = self.get_cred()
       self.cred["schema"] = self.schema
       self.cur = self.connect(self.cred)
       self.target_table = target_table.upper()
       self.target_stg_table = f"{target_table}_STG"

   def get_cred(self):
       return {
           "user": getenv("dwh_username","input" ),
           "password": getenv("dwh_password", "input"),
           "account": getenv("dwh_account", "input"),
           "role": getenv("dwh_role",'X_BAS_BI_USER'),
           "database": getenv("dwh_database", "DWH"),
           "warehouse":getenv('warehouse','COMPUTE_WH')
       }

   def connect(self, cred):
       self.conn = connector.connect(**cred)
       return self.conn.cursor()

   def get_columns(self, tablename):
       result = self.cur.execute(
           f"""SELECT
       COLUMN_NAME , DATA_TYPE
       FROM INFORMATION_SCHEMA.COLUMNS
       WHERE TABLE_SCHEMA = '{self.schema}'
       AND TABLE_NAME = '{tablename}'"""
       ).fetchall()

       return {x[0]: x[1] for x in result}

   def create_table(self, col_definition, table=None):

       if table is None:
           table = self.target_table

       return self.cur.execute(
           f"CREATE TABLE IF NOT EXISTS {table} ({col_definition});"
       ).fetchone()

   def add_column(self, colname, datatype):
       return self.cur.execute(
           f"ALTER TABLE {self.target_table} ADD {colname} {datatype}"
       ).fetchone()

   def put_file(self, path):
       return self.cur.execute(
           f"put file://{path} @{self.target_stg_table};"
       ).fetchone()

   def create_stg_table(self):
       return self.cur.execute(
           f"CREATE OR REPLACE STAGE {self.target_stg_table} file_format = (type = 'PARQUET');"
       ).fetchone()

   def close(self):
       self.cur.close()
       self.conn.close()

   def drop_table(self, table):
       return self.cur.execute(f"DROP TABLE IF EXISTS {table}").fetchone()

   def table_def_update(self):
       target_cols = self.get_columns(self.schema, self.target_table)
       stg_cols = self.get_columns(self.schema, self.target_stg_table)

       new_cols = set(target_cols.keys()) - set(stg_cols.keys())
       for new_col in new_cols:
           self.add_column(new_col, target_cols[new_col])

   def overwrite_target_table(self, col_definition, cols):

       swap_table = f"swap_{self.target_table}"
       temp_table = f"temp_{self.target_table}"

       self.drop_table(swap_table)
       self.create_table(col_definition, swap_table)
       self.copy_from_stage(cols, swap_table)
       self.drop_table(temp_table)
       self.swap_table(self.target_table, swap_table, temp_table)

   def swap_table(self, target: str, source: str, temp: str) -> str:
       result = self.cur.execute(f"ALTER TABLE {target} RENAME TO {temp};").fetchone()
       result1 = self.cur.execute(
           f"ALTER TABLE {source} RENAME TO {target};"
       ).fetchone()
       return {result, result1}

   def copy_from_stage(self, cols, tablename):

       statement = ""
       for col in cols:
           if col == "DWH_TIMESTAMP":
               statement += f'to_timestamp($1:"DWH_TIMESTAMP"::int, 9),'
           else:
               statement += f'$1:"{col}",'

       return self.cur.execute(
           f"""COPY INTO {tablename} FROM (SELECT {statement[:-1]} FROM @{self.target_stg_table})"""
       ).fetchone()

   def get_count(self, table=None) -> int:

       if table is None:
           table = self.target_table

       return self.cur.execute(
           f"SELECT COUNT(*) FROM {self.schema}.{table};"
       ).fetchone()[0]

   def get_max(self, column, table=None) -> int:

       if table is None:
           table = self.target_table

       return self.cur.execute(
           f"SELECT MAX({column}) FROM {self.schema}.{table};"
       ).fetchone()[0]

   def merge_table(self, cols, unique_columns):

       on = ",".join([f'TARGET."{col}" = SOURCE."{col}"' for col in unique_columns])
       update = ",".join([f'TARGET."{col}" = SOURCE."{col}"' for col in cols])
       columns = ",".join([f'"{col}"' for col in cols])
       values = ",".join([f'SOURCE."{col}"' for col in cols])

       stage_statement = ""
       for col in cols:
           if col == "DWH_TIMESTAMP":
               stage_statement += (
                   f'to_timestamp($1:"DWH_TIMESTAMP"::int, 9) AS "DWH_TIMESTAMP",'
               )
           else:
               stage_statement += f'$1:"{col}" AS "{col}",'

       stage_statement = f"SELECT {stage_statement[:-1]} FROM @{self.target_stg_table}"

       sql_statement = f"""
       MERGE INTO
           {self.schema}.{self.target_table} AS TARGET
       USING
           ({stage_statement}) AS SOURCE
       ON {on}
       WHEN MATCHED THEN
       UPDATE SET {update}
       WHEN NOT MATCHED THEN
       INSERT ({columns})
       VALUES ({values})
       """

       return self.cur.execute(sql_statement).fetchone()[0]

ETLMode = {"full_refresh": "full_refresh", "incremental": "incremental"}
ETLMode = namedtuple("x", ETLMode.keys())(*ETLMode.values())

In [28]:
def source_to_parquet(
   tablename,
   filename,
   db_conn,
   etl_mode,
   path,
   sheet=None,
   skiprows=0,
   excel=False,
   offset=0,
):
   _file = os.path.join(path, filename)

   if excel == True:
       df = get_excel_df(_file,sheet,skiprows)       
   else:
       df = pd.read_csv(_file, skiprows=skiprows, dtype=str)

   if etl_mode != ETLMode.full_refresh:
       if df.shape[0] + offset < db_conn.get_count():
           raise Exception(
               f"DWH table:{tablename} has more count than source table:{filename}"
           )

   df.columns = [col.upper() for col in df.columns]
   df.insert(0, "ID", df.index.values + offset + 2 + skiprows)
   df["DWH_TIMESTAMP"] = datetime.now().replace(microsecond=0)

   df.dropna(how="all")

   df.to_parquet(
       path + f"{tablename}_parquet.gzip",
       compression="gzip",
       engine="fastparquet",
       index=False,
   )

   return get_col_def(df.columns), df.columns



def df_to_parquet(df, tablename, path = report_folder__cdm, offset = 0, skiprows = 0):

   df.columns = [col.upper() for col in df.columns]
   df.insert(0, "ID", df.index.values + offset + 2 + skiprows)
   df["DWH_TIMESTAMP"] = datetime.now().replace(microsecond=0)

   df.dropna(how="all")

   df.to_parquet(
       path + f"{tablename}_parquet.gzip",
       compression="gzip",
       engine="fastparquet",
       index=False,
   )

   return get_col_def(df.columns), df.columns



def get_col_def(cols):

   col_definition = ""
   for col in cols:
       if col == "DWH_TIMESTAMP":
           col_definition += f'"{col}" TIMESTAMP,'
       elif col == "ID":
           col_definition += f'"{col}" INT,'
       else:
           col_definition += f'"{col}" VARCHAR,'

   return col_definition[:-1]



def get_excel_df(_file,sheet,skiprows):
  
   wb = load_workbook(_file,read_only=True,data_only=True,keep_links=False)
   ws = wb[sheet]
   data = ws.values

   for _ in range(skiprows):
       next(data)

   cols = next(data)

   counter = dict(Counter(cols).items())
   for _key, _value in counter.items():
       counter[_key] = {'encounter':0,'cnt':_value}

   new_cols = []

   for col in cols:
       if counter[col]['encounter'] > 0:
           new_cols.append(f"{col}.{counter[col]['encounter']}")
       else:
           new_cols.append(col)

       counter[col]['encounter'] += 1

   df = pd.DataFrame(data,dtype=str,columns=new_cols)
   wb.close()
   df = df.loc[:,df.columns.values != None]
   df = df.loc[:,~df.columns.str.contains("None")]

   return df


In [29]:
def csv_upload__to_snowflake(schema, tablename, path, filename, mode):
  
   sf_loader = SnowflakeLoader(schema, tablename)
   print("====Connected to snowflake====")

   col_def, cols = source_to_parquet(
       tablename = tablename,
       filename = filename,
       db_conn = sf_loader,
       etl_mode = ETLMode.full_refresh, #incremental #full_refresh
       sheet=None,
       path= path,
       skiprows=0,
       excel=False,
       offset=0,
   )

   print(f"====Converted csv/excel {filename} to parquet====")

   try:
       print(1)
       sf_loader.create_table(col_def)
       print(2)
       sf_loader.create_stg_table()
       print(3)
       sf_loader.put_file(os.path.join(path, f"{tablename}_parquet.gzip"))
       print(4)

       if mode == ETLMode.full_refresh:
           print(5)
           sf_loader.overwrite_target_table(col_def, cols)
       elif mode == ETLMode.incremental:
           print(6)
           sf_loader.merge_table(cols, unique_key.split(","))
       print(f"====Loaded the {tablename} to DWH====")
   except Exception as e:
       logging.exception(e)
   finally:
       os.remove(path + f"{tablename}_parquet.gzip")
       sf_loader.close()


In [30]:
for file_path in raw_lists:
    file_name = file_path.split("/")[-1] 
    table_name = file_name.split('.')[0]
    print(f'\nIngest table {table_name} to snowflake')

    csv_upload__to_snowflake(
        schema = 'CDM_DATA'
    , tablename = table_name
    , path = report_folder__cdm
    , filename = file_name
    , mode = ETLMode.full_refresh
    )


INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 2.7.12, Python Version: 3.9.13, Platform: Windows-10-10.0.19044-SP0
INFO:snowflake.connector.connection:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.



Ingest table cdm__quarterly_data to snowflake


INFO:snowflake.connector.cursor:query: [CREATE TABLE IF NOT EXISTS CDM__QUARTERLY_DATA ("ID" INT,"YEAR" VARCHAR,"WEEK" V...]


====Connected to snowflake====
====Converted csv/excel cdm__quarterly_data.csv to parquet====
1


INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE OR REPLACE STAGE cdm__quarterly_data_STG file_format = (type = 'PARQUET')...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [put file://c:/cdm/cdm__quarterly_data_parquet.gzip @cdm__quarterly_data_STG;]


...

3


INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [DROP TABLE IF EXISTS swap_POINT_TARGET]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [CREATE TABLE IF NOT EXISTS swap_POINT_TARGET ("ID" INT,"YEAR" VARCHAR,"QUARTER" ...]


4
5


INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [COPY INTO swap_POINT_TARGET FROM (SELECT $1:"ID",$1:"YEAR",$1:"QUARTER",$1:"REGI...]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [DROP TABLE IF EXISTS temp_POINT_TARGET]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [ALTER TABLE POINT_TARGET RENAME TO temp_POINT_TARGET;]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.cursor:query: [ALTER TABLE swap_POINT_TARGET RENAME TO POINT_TARGET;]
INFO:snowflake.connector.cursor:query execution done
INFO:snowflake.connector.connection:closed
INFO:snowflake.connector.connection:No async queries seem to be running, deleting session


====Loaded the point_target to DWH====
